# このスライドへのアクセス方法

1. 「nbviewer.jupyter.org」というサイトへアクセス
2. 「mrsekut」で検索
3. 「RealWorldHTTP > myproduct > slide > HTTP2.ipynb」
4. 右上のプレゼントマークでスライドっぽく表示
5. スペースキーで順番に移動

[リンク](http://nbviewer.jupyter.org/github/mrsekut/RealWorldHTTP/blob/master/myproduct/slide/HTTP2.ipynb)


# HTTP/2.0について

丸末皓太

# 自己紹介

### 丸末皓太
- 大阪府立大学(一応2回生)
- 休学中
- 府大生向けの「Opulency」というアプリを作った
- 現在はtryangle社でばりばりアプリ開発してる卍卍
- 最近興味あること
    - Typescript: ReactNative
    - Haskell: 関数型プログラミング
    - Go: ネットワーク, プログラミング言語そのもの
    - Rust: OSなどの低レイヤ
    - テスト  



![mrsekut](https://github.com/mrsekut/myStorage/blob/master/img/mrsekut_circle.png?raw=true)

### SNS
- blog: [mrsekutの備忘録: http://mrsekut.site/](http://mrsekut.site/)
- twitter: [@mrsekut](https://twitter.com/mrsekut)
- github: [mrsekut](https://github.com/mrsekut)

# 発表のながれ
★あとで修正
1. 発表の目的
1. 従来のHTTP
1. HTTP/2の概要
1. ためしてみる
1. なんか
1. なんか


# 発表のながれ
★あとで修正

1. __発表の目的__


# 発表の目的

- 浅く広く紹介します
- キーワードを拾っていってください

★あとで修正


1. 発表の目的
1. __従来のHTTP__
1. HTTP/2の概要
1. なんか
1. なんか
1. なんか


# 従来のHTTP

- リクエストとレスポンス
- HTTP/1.1
    - 1997年に策定

## HTTP1.1の課題

- リクエストの多重化
    - 
    - 1RTTで1リクエスト/1レスポンスしかできない
- パイプライン化(課題か？
    - 同時に複数のリクエストを送れるようになった
    - しかし、同じ順序でレスポンスしないといけない
- HoLブロッキング問題

- なので、オーバーヘッドが生じる

★あとで修正


1. 発表の目的
1. 従来のHTTP
1. __HTTP/2の概要__
1. ためしてみる
1. なんか
1. なんか



# HTTP/2の概要

## 誕生の背景

- アクセスの増加
- 16年ぶりの改訂
- GoogleのSPDYから生まれた

- LTEなどの普及によりエンドユーザーの通信速度は上昇したが、サーバー側のページロードの速度はそれに比例せず1.6Mbpsほどで頭打ちしていた
- 

## HTTP/2の特徴

- ヘッダの圧縮
- サーバープッシュ
- リクエストの優先度制御
- バイナリベース
- シングルコネクション
- マルチプレックス

### ヘッダの圧縮


- ヘッダー圧縮
    - 従来の圧縮に加え、ヘッダ部分の圧縮ができる
    - HPACK
        - バイナリ形式
        - 高頻度で用いられるヘッダーのキーと値を組みにした辞書
        - 動的に辞書を更新し、2回目移行はインデックスを用いる
        - キーや値の文字列をハフマン符号化によって圧縮
        - 差分のみを送信する


- HTTP/1.1のヘッダは大きい
    - リクエストは最低でも300Byte程度
    - Cookieなどで更に増える

### フロー制御

    - ストリームごとにフロー制御が可能


### 多重化

- 多重化(どれかの項目とかぶっているかも)
   - 同時に100以上のリクエストを送信可能
   - 任意のタイミングでリクエスト送信可能

   - レスポンスの順序に制限なし
   - 1つの接続上にストリームと呼ばれる仮想的な双方向シーケンスを作る(するとどうなる？)
   - 複数並列に扱うことができる
   - HTTPパイプライン問題を解決
   - ストリームはサーバ側でもクライアント側でも開始可能
   - ストリームにはIDが振られ、クライアント側は奇数、サーバ側は偶数番号を割り当て、重複を防いでいる


### サーバープッシュ

- 今後の利用のために追加のリクエストがなくとも、追加のリソースをクライアント側に送信することができる
- 0RTTにすることができる
- サーバーがクライアントの発行するリクエストを予測してレスポンスをプッシュ


### リクエストの優先度制御

- クライアントがサーバーにレスポンス優先度を指定できる
- 優先度を付けることでユーザ体感速度を大幅に向上させることができる
- 重み付け
    - 値に比例したバンド幅の配分★
- 依存関係
    - 依存しているレスポンス先に送信
    - グループ化にも利用
    
##### 参考
39~
https://www.slideshare.net/kazuho/http-58452175

- 「重み付け」と「依存関係」の2種類の制御手法の組み合わせで優先度を制御している
- サーバーは指定された優先度を「参考にする」★

### バイナリベース


  - サーバーがパースを簡単に行えるようになり、よりコンパクトに、そしてエラーを減らすことができる
  - 脆弱性を防ぐ
    - HTTP Response Splitting Attack
    - レスポンス分割攻撃
  - 転送データ量の低減
  - 全てのデータは「フレーム」に分解して送受信


### シングルコネクション

  - ウェブサイトを読み込むためにサーバーに接続する回数はたったの一度
  - TCPコネクションを張るために必要なラウンドトリップの回数を減らす




### マルチプレックス

  - 同じ接続で同時期に複数のリクエストを送信することができる
  - 1.1では別の送信が完了するまで待たないといけなかった


# 試してみる

In [3]:
!curl -v http://example.com

* Rebuilt URL to: http://example.com/
*   Trying 93.184.216.34...
* TCP_NODELAY set
* Connected to example.com (93.184.216.34) port 80 (#0)
> GET / HTTP/1.1
> Host: example.com
> User-Agent: curl/7.54.0
> Accept: */*
> 
< HTTP/1.1 200 OK
< Cache-Control: max-age=604800
< Content-Type: text/html; charset=UTF-8
< Date: Tue, 28 Aug 2018 12:15:55 GMT
< Etag: "1541025663+ident"
< Expires: Tue, 04 Sep 2018 12:15:55 GMT
< Last-Modified: Fri, 09 Aug 2013 23:54:35 GMT
< Server: ECS (oxr/8313)
< Vary: Accept-Encoding
< X-Cache: HIT
< Content-Length: 1270
< 
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        

## chromeで確認する方法


# 対応

# さいごに

- もうちょい詳細にしてブログにまとめます

# 参考文献

- Real World HTTP
- https://ja.wikipedia.org/wiki/HTTP/2
- https://blog.redbox.ne.jp/http2-cdn.html
- http://www.seojapan.com/blog/everyone-moving-http2
